In this demo we create another cluster and use the built-in `pgbench` support of the CNPG plugin to test failover and measure the time it takes. 

In [ ]:
%%bash
kubectl create -f resources/cluster.pgbench.yml

Wait until the cluster is operational

In [ ]:
%%bash
kubectl cnpg status demo

Initialise the pgbench schema, using the `pgbench` command of the CNPG plugin

In [ ]:
%%bash
kubectl cnpg pgbench demo -- -i -d pgbench

This creates a Kubernetes job that runs the `pgbench` command with the specified parameters.

In [ ]:
%%bash
kubectl get jobs -l pbBenchJob=demo

We cannot simply run `pgbench` and perform a failover, because it will terminate as soon as it loses the connection to the primary server. We want to create a little wrapper script for , which will restart `pgbench` once that happens, and store it in a `ConfigMap`.

We also modify the `pgbench_history` table slightly by adding a column that is automatically populated with the server IP address. This will help us capture the exact moment when the client reconnects to the newly promoted primary.

Finally, once `pgbench` completes, we will run a query to calculate the failover latency.

In [ ]:
%%bash
kubectl apply -f resources/configmap.scripts.yml

We will simulate a "hard" failure, which is the worst case scenario, by stopping the worker node on which the primary container is running. For that we need to determine that node.

In [ ]:
%%bash
kubectl cnpg status demo

Now we can create a `Job` that will run the wrapper script.

In [ ]:
%%bash
kubectl create -f resources/job.pgbench-with-wrapper.yml

Let's stop the worker on which the primary node is running, as determined from the `cnpg status` output above. Edit the node name as approriate in the command below.

This demo assumes the Kubernetes cluster is provided by [`kind`](https://kind.sigs.k8s.io/); use the method appropriate for your particular environment.

In [ ]:
%%bash
podman stop demo-worker3

Let's see what the cluster status is:

In [ ]:
%%bash
kubectl cnpg status demo

Wait for the job to complete

In [ ]:
%%bash
kubectl get job -l app=failover-test

Check the job log:

In [ ]:
%%bash
kubectl logs -l app=failover-test

Delete the test cluster

In [ ]:
%%bash
kubectl delete cluster demo